In [ ]:
import atlite
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
from rasterio.plot import show
from atlite.gis import shape_availability, ExclusionContainer

import os
from matplotlib.gridspec import GridSpec
import seaborn as sns
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import cartopy.crs as ccrs
from cartopy.crs import PlateCarree as plate
import cartopy.io.shapereader as shpreader

import logging
import warnings

warnings.simplefilter('ignore')
logging.captureWarnings(False)
logging.basicConfig(level=logging.INFO)

plt.rcParams['figure.figsize'] = [7, 7]
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
countries = ['Serbia', 'Bosnia and Herz.', 'Montenegro']
shapes = world[world.name.isin(countries)].set_index('name')

shapes.plot();

In [ ]:
bounds = shapes.cascaded_union.buffer(1).bounds
cutout = atlite.Cutout('DRB_2020', module='era5', bounds=bounds, time=slice('2020-01-01', '2020-12-31'))

In [ ]:
fig, ax = plt.subplots(figsize=(20,20))
shapes.plot(ax=ax)
cutout.grid.plot(ax=ax, edgecolor='grey', color='None')

In [ ]:
CORINE = 'corine.tif'
excluder = ExclusionContainer()
excluder.add_raster(CORINE, codes=(1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,30,34,35,36,37,38,39,40,41,42,43,44,45))

In [ ]:
serbia = shapes.loc[['Serbia']].geometry.to_crs(excluder.crs)
bosnia = shapes.loc[['Bosnia and Herz.']].geometry.to_crs(excluder.crs)
montenegro = shapes.loc[['Montenegro']].geometry.to_crs(excluder.crs)

In [ ]:
masked, transform = shape_availability(bosnia, excluder)
eligible_share = masked.sum() * excluder.res**2 / bosnia.geometry.item().area

In [ ]:
bosnia.geometry.item().area

In [ ]:
fig, ax = plt.subplots()
ax = show(masked, transform=transform, cmap='Greens', ax=ax)
bosnia.plot(ax=ax, edgecolor='k', color='none',figsize=(15, 25))
ax.set_title(f'Eligible area (green) {eligible_share * 100:2.4f}%');
plt.savefig('RS_2020.png', dpi=300)

In [ ]:
fig, ax = plt.subplots()
ax = show(masked, transform=transform, cmap='Greens', ax=ax)
bosnia.plot(ax=ax, edgecolor='k', color='None')
cutout.grid.to_crs(excluder.crs).plot(edgecolor='grey', color='None', ax=ax, ls=':')
ax.set_title(f'Eligible area (green) {eligible_share * 100:2.2f}%');

In [ ]:
A = cutout.availabilitymatrix(shapes, excluder)
A

In [ ]:
fig, ax = plt.subplots(figsize=(20,20))
A.sel(name='Bosnia and Herz.').plot(cmap='Greens')
shapes.loc[['Bosnia and Herz.']].plot(ax=ax, edgecolor='k', color='None')
cutout.grid.plot(ax=ax, color='None', edgecolor='grey', ls=':')

In [ ]:
float(A.sel(name='Bosnia and Herz.')[8,12])

In [ ]:
fg = A.plot(row='name', col_wrap=3, cmap='Greens',figsize=(15, 6))
fg.set_titles('{value}')
for i, c in enumerate(shapes.index):
    shapes.loc[[c]].plot(ax=fg.axes.flatten()[i], edgecolor='k', color='None')
plt.savefig('DRB_riparians_2020', dpi=300)   

In [ ]:
cap_per_sqkm = 1.7
area = cutout.grid.set_index(['y', 'x']).to_crs(3035).area / 1e6
area = xr.DataArray(area, dims=('spatial'))

capacity_matrix = A.stack(spatial=['y', 'x']) * area * cap_per_sqkm

In [ ]:
cutout.prepare()
pv = cutout.pv(matrix=capacity_matrix, panel=atlite.solarpanels.CdTe,
               orientation='latitude_optimal', index=shapes.index)    

In [ ]:
cutout.prepared_features

In [ ]:
pv.to_pandas().div(1e3).plot(ylabel='Solar Power [GW]', ls='--', figsize=(10, 4));
plt.savefig('Solar_Power_[GW]_2020.png', dpi=300)

In [ ]:
pv.to_pandas().to_csv('/Users/solar_cf_2020.csv')

In [ ]:
wind = cutout.wind(matrix=capacity_matrix, turbine='Siemens_SWT_2.3_108',
               index=shapes.index)

In [ ]:
cutout.prepared_features

In [ ]:
cutout.wind(turbine='Siemens_SWT_2.3_108').plot()

In [ ]:
wind.to_pandas().div(1e3).plot(ylabel='Wind Power [GW]', ls='--', figsize=(10, 4));
plt.savefig('Wind_Power_[GW]_2020.png', dpi=300)

In [ ]:
wind.to_pandas().to_csv('/Users/wind_cf_2020.csv')